# Downloading single files or repositories 

### Notes

Links:
-  "https://github.com/search?p=1&q=extension%3Aipynb+nbformat_minor&type=Code&utf8=%E2%9C%93"

### Import

In [1]:
# imports
import requests
import re
import os
import json

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from git import Repo

# import git    # add 'git' module through anaconda navigator + pip install gitpython

### Functions

In [6]:
def try_link_getter(ul_list):
    """
    Explanation function
    """
    links = []
    for ul in ul_list:
        for line in ul:
            try:
                links.append(line.a.get('href'))
            except:
                pass
    return links

def download_(url):
    """
    Explanation function
    """
    name = url.split('/')[-1]
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    convert = soup.get_text()
    try: 
        json_data = json.loads(convert)
        #json_data = json.loads(convert.replace(" ","").replace("\n",""))
        cwd = os.getcwd()
        with open(cwd+'\\single_files\\'+name, "w") as outfile:
            json.dump(json_data,outfile)
    except Exception as e:
        print(e,url)
#         if 'Extra data' in e:
#             print("404 Page not found: ",url)
# #         else:
#             print("Downloading failed for url: ",url)


def nbviewer_to_download(nbviewer):
    """
    Explantion function
    """
    try:
        m = re.search(".org/github", nbviewer)
        if m != None:
            prefix = 'https://raw.githubusercontent.com'
            download_url = prefix+nbviewer[m.end():].replace('/blob','')
            download_(download_url)
        elif re.search(".org/url/", nbviewer) != None:
            m = re.search(".org/url", nbviewer)
            download_url = 'http://'+nbviewer[m.end()+1:]
            download_(download_url)
    except Exception as e:
        print(e)
        print(nbviewer)


def clone_repos(repo_name,url,folder):
    """
    Explenation function
    """
    cwd = os.getcwd()
    try:
        repo_dir = cwd+'\\'+folder+'\\'+repo_name
        correct_url = '/'.join(url.split('/')[:5]) # makes sure it is a url to a repository (not blob/master/readme etc.) 
        Repo.clone_from(correct_url,repo_dir)
    except Exception as e:
#         print(e)
        print('Failed for:',repo_name,url)

### Webscraping <i>A gallery of interesting Jupyter Notebooks</i>

In [3]:
url = "https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks"

r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [4]:
body = soup.find_all("div", {"class": "markdown-body"})[0] # using [0] at the end since there is only 1 result of a div with class 'markdown-body'
ul_list = body.find_all('ul')

links = try_link_getter(ul_list)
github_repos = [link for link in links if 'nbviewer' not in link and 'github.com' in link]
github_single_files = [url for url in links if 'nbviewer' in url and 'github' in url]

# uitleggen waarom nbviewer.gist exclude from list!!
# nbviewer_list = [link for link in links if 'nbviewer' in link]

# TEST
# len(links),sorted(links)

### Downloading single files from url's

In [5]:
# create folder in which to put all the files (if it does not exist)
if not os.path.exists('single_files'):
    os.makedirs('single_files')

In [6]:
# ALL SINGLE GITHUB FILES (NBVIEWER)
for url in tqdm_notebook(github_single_files):
    nbviewer_to_download(url)
    
# Extra data: line 1 column 4 (char 3) => blank page with '404: Not Found'

# Verbetering zit er geen '.ipynb' in de string dan is er sprake van een repository

Expecting ',' delimiter: line 2901 column 10 (char 1540277) https://raw.githubusercontent.com/jrjohansson/scientific-python-lectures/master/Lecture-4-Matplotlib.ipynb
Expecting ',' delimiter: line 298 column 10 (char 22663) https://raw.githubusercontent.com/ganeshutah/Jove/master/notebooks/driver/Drive_Jove_Gallery_Examples.ipynb
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/mwaskom/seaborn/master/examples/linear_models.ipynb
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/leig/Applied-Predictive-Modeling-with-Python/tree/master/notebooks/
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/ledeprogram/courses/tree/master
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/thsant/scipy4cv
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/apache/incubator-singa/master/doc/en/docs/notebook/index.ipynb
Extra data: line 1 column 4 (char 3) https://raw.githubusercontent.com/ketch/Hyper

### Clone repositories from url's

In [5]:
# create folder in which to put all the repositories (if it does not exist)
if not os.path.exists('repos'):
    os.makedirs('repos')

In [7]:
# ALL GITHUB REPOS
for url in tqdm_notebook(github_repos):
    try:
        folder_name = url.split('/')[4]
        clone_repos(folder_name,url,'repos')
    except:
        print(url) # one url is not a link to a repository but to the user's page

Failed for: Python https://github.com/Tanu-N-Prabhu/Python/blob/master/Predicting_PewDiePie's_daily_subscribers_using_Machine_Learning_.ipynb
http://amueller.github.com/sklearn_tutorial
Failed for: Python https://github.com/Tanu-N-Prabhu/Python/blob/master/Pandas/Pandas_DataFrame.ipynb
https://github.com/lgiordani
Failed for: MetodosNumericosAvanzados https://github.com/dpsanders/MetodosNumericosAvanzados



In [12]:
duplicates_repo_url = [repo for repo in github_repos if github_repos.count(repo)>1]
duplicates_repo_url

['https://github.com/dpsanders/MetodosNumericosAvanzados',
 'https://github.com/dpsanders/MetodosNumericosAvanzados']

In [13]:
len(github_repos),sorted(github_repos)

(63,
 ['http://amueller.github.com/sklearn_tutorial',
  'https://github.com/BuzzFeedNews/2014-08-st-louis-county-segregation',
  'https://github.com/JWarmenhoven/DBDA-python',
  'https://github.com/Neuraxio/Kata-Clean-Machine-Learning-From-Dirty-Code',
  'https://github.com/Qiskit/qiskit-tutorial/blob/master/index.ipynb',
  'https://github.com/ResearchComputing/Meetup-Fall-2013',
  'https://github.com/SciRuby/sciruby-notebooks',
  'https://github.com/Tanu-N-Prabhu/Python/blob/master/Pandas/Pandas_DataFrame.ipynb',
  "https://github.com/Tanu-N-Prabhu/Python/blob/master/Predicting_PewDiePie's_daily_subscribers_using_Machine_Learning_.ipynb",
  'https://github.com/TomAugspurger/PyDataSeattle',
  'https://github.com/agmarrugo/sensors-actuators',
  'https://github.com/barbagroup/AeroPython',
  'https://github.com/bloomberg/bqplot',
  'https://github.com/calebmadrigal/FourierTalkOSCON',
  'https://github.com/cfangmeier/Small/blob/9037dbd6d7e5843a60d7a92c6e4a713b3b685a74/Imagination.ipynb',
 